In [1]:
!pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets list

ref                                                            title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
thedrcat/daigt-v2-train-dataset                                DAIGT V2 Train Dataset                              29MB  2023-11-16 01:38:36           1021        113  1.0              
muhammadbinimran/housing-price-prediction-data                 Housing Price Prediction Data                      763KB  2023-11-21 17:56:32           3741         77  1.0              
carlmcbrideellis/llm-7-prompt-training-dataset                 LLM: 7 prompt training dataset                      41MB  2023-11-15 07:32:56           1381        106  1.0              
thedrcat/daigt-proper-train-dataset                            DAIGT P

In [6]:
!kaggle datasets download -d arbethi/diabetic-retinopathy-level-detection

100% 9.66G/9.66G [01:20<00:00, 157MB/s]
100% 9.66G/9.66G [01:20<00:00, 129MB/s]


In [7]:
!unzip -q /content/diabetic-retinopathy-level-detection.zip

In [8]:
imageSize = [299, 299]
trainPath = r'/content/preprocessed dataset/preprocessed dataset/train'
testPath = r'/content/preprocessed dataset/preprocessed dataset/test'

In [9]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import matplotlib.pyplot as plt
import numpy as np

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
training_set = train_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/training',
                                      target_size=(299, 299),
                                      class_mode='categorical',
                                      batch_size=32)
test_set = test_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/testing',
                                    target_size=(299, 299),
                                      class_mode='categorical',
                                      batch_size=32)

Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


In [12]:
training_set.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}

In [13]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet', include_top=False)

83683744/83683744 [==============================] - 0s 0us/step


In [14]:
for layer in xception.layers:
  layer.trainable = False

In [15]:
x = Flatten()(xception.output)

In [16]:
prediction = Dense(5, activation='softmax')(x)

In [17]:
model = Model(inputs=xception.input, outputs=prediction)

In [18]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [19]:
batch_size = 32
steps_per_epoch = len(training_set) // batch_size

In [20]:
model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=30,
  steps_per_epoch=len(training_set) // batch_size,

  validation_steps=len(test_set)
)

<ipython-input-20-b8d67da5f866>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/30
3/3 [==============================] - 116s 48s/step - loss: 10.3627 - accuracy: 0.3333 - val_loss: 9.5256 - val_accuracy: 0.6744
Epoch 2/30
3/3 [==============================] - 98s 46s/step - loss: 13.6696 - accuracy: 0.4896 - val_loss: 5.1921 - val_accuracy: 0.6894
Epoch 3/30
3/3 [==============================] - 96s 46s/step - loss: 5.5810 - accuracy: 0.5521 - val_loss: 4.6602 - val_accuracy: 0.5831
Epoch 4/30
3/3 [==============================] - 98s 46s/step - loss: 4.6292 - accuracy: 0.5521 - val_loss: 5.1874 - val_accuracy: 0.6921
Epoch 5/30
3/3 [==============================] - 96s 46s/step - loss: 6.0952 - accuracy: 0.6979 - val_loss: 4.3979 - val_accuracy: 0.7057
Epoch 6/30
3/3 [==============================] - 97s 47s/step - loss: 4.7465 - accuracy: 0.6771 - val_loss: 3.1828 - val_accuracy: 0.6185
Epoch 7/30
3/3 [==============================] - 98s 47s/step - loss: 4.7741 - accuracy: 0.5417 - val_loss: 2.6210 - val_accuracy: 0.6049
Epoch 8/30
3/3 [========

In [22]:
model.save('updated-xception-diabetic-retinopathy.h5')